In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Загрузка даних
data = pd.read_csv('GlobalYouTubeStatistics.csv', delimiter=',', encoding='latin1')

# ВИдалення нульових значень
data = data.dropna()

# Перетворення категоріальних даних на float
label_encoder = LabelEncoder()
data['uploads'] = label_encoder.fit_transform(data['uploads'])
data['uploads'] = data['uploads'].astype(float)
data['Country'] = label_encoder.fit_transform(data['Country'])
data['Country'] = data['Country'].astype(float)
data['channel_type'] = label_encoder.fit_transform(data['channel_type'])
data['channel_type'] = data['channel_type'].astype(float)

# Розбиття даних
X = data[['uploads', 'Country']][:500]
y = data['channel_type'][:500]

# Розділення на навчальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Створення чотирьох різних моделей
decision_tree_model = DecisionTreeClassifier(max_depth=3)
random_forest_model = RandomForestClassifier(n_estimators=50, random_state=42)
svm_model = SVC(probability=True)
logistic_regression_model = LogisticRegression(max_iter=10000)

# Створення паралельного гетерогенного ансамблю
parallel_het_ensemble = VotingClassifier(estimators=[
    ('decision_tree', decision_tree_model),
    ('random_forest', random_forest_model),
    ('svm', svm_model),
    ('logistic_regression', logistic_regression_model)
], voting='soft')

# Тренування паралельного гетерогенного ансамблю
parallel_het_ensemble.fit(X_train, y_train)

# Прогнозування на тестових даних
y_pred_parallel = parallel_het_ensemble.predict(X_test)

# Оцінка точності паралельного гетерогенного ансамблю
accuracy_parallel = accuracy_score(y_test, y_pred_parallel)
print(f"Accuracy of the Parallel Heterogeneous Ensemble: {accuracy_parallel}")

Accuracy of the Parallel Heterogeneous Ensemble: 0.35


In [15]:
from sklearn.model_selection import GridSearchCV

# Створення гетерогенного паралельного ансамблю
parallel_het_ensemble = VotingClassifier(estimators=[
    ('decision_tree', DecisionTreeClassifier()),
    ('random_forest', RandomForestClassifier()),
    ('svm', SVC(probability=True)),
    ('logistic_regression', LogisticRegression(max_iter=20000))
], voting='soft')

# Параметри для пошуку в кожній моделі
param_grid = {
    'decision_tree__max_depth': [3, 5],
    'random_forest__n_estimators': [50, 100],
    'svm__C': [0.1, 1],
    'logistic_regression__C': [0.1, 1]
}

# GridSearchCV для пошуку оптимальних параметрів
grid_search = GridSearchCV(parallel_het_ensemble, param_grid, cv=2)
grid_search.fit(X_train, y_train)

# Вивід найкращих параметрів
best_params = grid_search.best_params_
print(f"Best Hyperparameters for Parallel Heterogeneous Ensemble: {best_params}")

# Використання моделі з найкращими параметрами
best_parallel_het_ensemble = grid_search.best_estimator_

# Тренування ансамблю на оптимальних параметрах
best_parallel_het_ensemble.fit(X_train, y_train)

# Точність
accuracy = best_parallel_het_ensemble.score(X_test, y_test)
print(f"Accuracy for the Best Parallel Heterogeneous Ensemble: {accuracy}")

D:\Software\Anaconda\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


Best Hyperparameters for Parallel Heterogeneous Ensemble: {'decision_tree__max_depth': 3, 'logistic_regression__C': 1, 'random_forest__n_estimators': 50, 'svm__C': 1}
Accuracy for the Best Parallel Heterogeneous Ensemble: 0.35


In [17]:
# Визначення важливості ознак для кожної моделі в гетерогенному ансамблі
for model_name, model in best_parallel_het_ensemble.named_estimators_.items():
    if hasattr(model, 'feature_importances_'):
        feature_importance = model.feature_importances_
    elif hasattr(model, 'coef_'):
        feature_importance = model.coef_[0]
    else:
        feature_importance = [0] * len(X_train.columns)
    
    feature_importance_dict = dict(zip(X_train.columns, feature_importance))
    
    # Вивід важливості ознак для кожної моделі
    print(f"Feature Importance for {model_name}:")
    for feature, importance in feature_importance_dict.items():
        print(f"{feature}: {importance}")
    print("\n")

Feature Importance for decision_tree:
uploads: 0.8292797242749822
Country: 0.1707202757250178


Feature Importance for random_forest:
uploads: 0.816028692197586
Country: 0.18397130780241405


Feature Importance for svm:
uploads: 0
Country: 0


Feature Importance for logistic_regression:
uploads: -0.0005763325989557184
Country: 0.12777404016380767


